In [56]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

In [57]:
data = pd.DataFrame()
data = pd.read_csv('Online_retail.csv',parse_dates=True)
data["InvoiceDate"]=pd.to_datetime(data["InvoiceDate"])
data = data[(data['Price']>0) & (data['Quantity']>0)] # eliminate Void transactions
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [58]:
# explore data
print(data.info(),data.describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530105 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      530105 non-null  object        
 1   StockCode    530105 non-null  object        
 2   Description  530105 non-null  object        
 3   Quantity     530105 non-null  int64         
 4   InvoiceDate  530105 non-null  datetime64[ns]
 5   Price        530105 non-null  float64       
 6   Customer ID  397885 non-null  float64       
 7   Country      530105 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 36.4+ MB
None             Quantity          Price    Customer ID
count  530105.000000  530105.000000  397885.000000
mean       10.542019       3.907652   15294.416882
std       155.523977      35.915652    1713.144421
min         1.000000       0.001000   12346.000000
25%         1.000000       1.250000   13969.0000

Calculate and Generate a table with recency, Frequency, Tenure, and Monetary Value.

In [59]:
def max_date(df):
    last_date = data["InvoiceDate"].max()+ dt.timedelta(days=1)
    return last_date

def get_tenure(x):
    return (max_date(x) - x.min()).days # How long was the first visit, can be change to first and last visit of a patient

def get_recency(x):
    return (max_date(x) - x.max()).days

Find values in the distribution using quantiles to separate the customers.

In [60]:
def table(df):
    rfm_table = df.groupby('Customer ID').agg({'InvoiceDate': [get_recency, get_tenure],
                                           'Invoice': 'count', 
                                           'Price': 'sum'})
    rfm_table.columns = ['Recency', 'Tenure', 'Frequency', 'Monetary']
    return rfm_table

quantiles = table(data).quantile(q = [0.20, 0.4, 0.6,.8 ]) #using quantile function to get value of quantiles
quantiles = quantiles.to_dict() # coverting into dictionary for easier mapping 
reference = pd.DataFrame(quantiles)

In [61]:
def R_score(val, param):
    if val <= quantiles[param][0.20]:   return 5
    elif val <= quantiles[param][0.40]: return 4
    elif val <= quantiles[param][0.60]: return 3
    elif val <= quantiles[param][0.80]: return 2
    else: return 1

def FM_score(val, param):
    if val <= quantiles[param][0.20]:   return 1
    elif val <= quantiles[param][0.40]: return 2
    elif val <= quantiles[param][0.60]: return 3
    elif val <= quantiles[param][0.80]: return 4
    else: return 5



In [62]:
rfm_table = table(data)
rfm_table['R'] = rfm_table['Recency'].apply(lambda x : R_score(x, 'Recency')) # giving R score 
rfm_table['F'] = rfm_table['Frequency'].apply(lambda x : FM_score(x, 'Frequency')) # giving F score
rfm_table['M'] = rfm_table['Monetary'].apply(lambda x : FM_score(x, 'Monetary'))

rfm_table['Group'] = rfm_table.R.map(str) + rfm_table.F.map(str) + rfm_table.M.map(str) 
# here we will make a new column of score which will store the score of group by adding RFM scores
rfm_table['Score'] = rfm_table[['R', 'F', 'M']].sum(axis = 1)

Create the group using individual scores from R,f,and m. This is just one method to create the groups. Another method would be to use the score.

In [63]:
# These values can be adjusted to customize the group.
def rfm_level(df):

    if  ((4<=df['R']<=5) and (df['Frequency']==1)):
        return ('New Customers')
    elif ((4<=df['R']<=5) and (df["F"]>=4)):
        return 'Champions'
    elif ((4<=df['R']<=5) and (2<=df["F"]<=3)):
        return 'Loyal Customers'
    
    elif ((2<=df['R']<=3) and (df["F"]==2)):
        return 'Potential'

    #elif ((df['R']==3) and (2<=df["F"]<=3)):
        #return 'Needs Attention'

    elif ((df['R']==2) and (1<=df["F"]<=3)):
        return 'At Risk'
    elif ((df['R']==2) and (4<=df["F"]<=5)):
        return 'Can\'t lose them'
    elif ((df['R']==1) and (4<=df["F"]<=5)):
        return "Lapse Frequent"
    else: 
        return 'Lapse'
        
# Create a new variable RFM_Level
rfm_table['rfm_level'] = rfm_table.apply(rfm_level, axis=1)

In [68]:
RFM_Marketing = rfm_table['rfm_level'] # This table can be used to implement Email Campaigns based on groups. 
RFM_Marketing.reset_index() # Additional information can be added for the use of marketing.

,Customer ID,rfm_level
0,12346.0,Lapse
1,12347.0,Champions
2,12348.0,At Risk
3,12349.0,Champions
4,12350.0,Lapse
...,...,...
4333,18280.0,Lapse
4334,18281.0,Lapse
4335,18282.0,Lapse
4336,18283.0,Champions


In [65]:
corr_matrix = rfm_table.corr()
pd.DataFrame(corr_matrix)
# Find any correlation the features created.

,Recency,Tenure,Frequency,Monetary,R,F,M,Score
Recency,1.000000,0.272267,-0.206126,-0.156613,-0.875745,-0.451359,-0.394569,-0.668674
Tenure,0.272267,1.000000,0.196580,0.154550,-0.139463,0.322117,0.355254,0.209648
Frequency,-0.206126,0.196580,1.000000,0.869930,0.264721,0.405880,0.390690,0.412706
Monetary,-0.156613,0.154550,0.869930,1.000000,0.196682,0.315647,0.332593,0.328536
R,-0.875745,-0.139463,0.264721,0.196682,1.000000,0.483481,0.431091,0.743518
F,-0.451359,0.322117,0.405880,0.315647,0.483481,1.000000,0.893420,0.924679
M,-0.394569,0.355254,0.390690,0.332593,0.431091,0.893420,1.000000,0.903956
Score,-0.668674,0.209648,0.412706,0.328536,0.743518,0.924679,0.903956,1.000000


Analyze the RFM table to understand customer segmentation.

In [66]:
rfm_level_agg = rfm_table.groupby('rfm_level').agg({
    'Recency': 'median',
    'Frequency': 'median','Tenure':'median',
    'Monetary': ['median','count']
}).round(1)
# Print the aggregated dataset
print(pd.DataFrame(rfm_level_agg))

                Recency Frequency Tenure Monetary      
                 median    median median   median count
rfm_level                                              
At Risk           112.0      14.0  162.0     57.5   432
Can't lose them   107.0      92.0  282.0    270.1   211
Champions          11.0     143.0  311.0    413.8  1101
Lapse              71.0      20.0  260.0     71.3  1625
Lapse Frequent    240.0      85.0  337.0    248.4    85
Loyal Customers    17.0      34.0  130.0     97.7   499
New Customers      16.0       1.0   16.0      2.1     9
Potential          75.0      22.0  134.0     66.8   376
